<a id = 'classic01'></a>

## Classical implementation of Problem


In [ ]:
from gurobipy import *
import numpy as np
import math as m
import time
import sys
from itertools import combinations as comb
from scipy.special import comb as comb2

In [ ]:
from functools import partial
from pycpd import rigid_registration

### Data Treatment

The information we initial have is the 3D coordinates of the points of our graphs.

In [ ]:
class Point( object ):
    def __init__( self, x, y, z, data ):
        self.x, self.y, self.z = x, y, z
        self.data = data

    def __str__ (self):
        return "Point of type %s. Coordinates %s %s %s" % (self.data, self.x, self.y, self.z)

In [ ]:
import csv

csv.register_dialect('myDialect',
delimiter = ',',
skipinitialspace=True)

In [ ]:
with open('input_BIAL.csv', 'r') as csvDataFile:
    csvReader = csv.reader(csvDataFile, dialect = 'myDialect')
    for row in csvReader:
        print(row)

In [ ]:
database = [] 

with open('input_BIAL.csv', 'r') as csvDataFile:
    csvReader = csv.reader(csvDataFile, dialect = 'myDialect')

    mol = -1
    molecule_type = ""
    idconf = -1
    
    for idx, row in enumerate(csvReader):
        if (row[0] != 'pdb'):
            # if the row does not have the title
            # then we have one of the molecules
            
            if (molecule_type != row[0]):
                # if the molecule in row[0] is different from 
                # the molecule stored in the variable "molecule_type"
                # identify the new molecule
                print ("Molecule", row[0])
                # update molecule 
                molecule_type = row[0]
                # create space in the list for the molecule
                database.append([])
                mol = mol+1
                idconf= -1
            # (else) we are in the same molecule
            
            if( idconf != int(row[1])):
                # if the conformation number is different 
                # from the one in variable idconf + 1
                # then identify this new conformation
                print("new conformation: ",row[1])
                # update the number of the conformation
                idconf = idconf+1
                # create space in the list for the next conformation 
                database[mol].append([])
            
            #simplify name of the types
            types=""
            if "H" in row[2]:
                types="H"
            if "P" in row[2]:
                types ="P"
            
            print("Adding the point",types,"with coordinates ",row[3],row[4],row[5])
            # add point to the corresponding molecule and conformation in the list
            database[mol][idconf].append(Point(float(row[3]), 
                                               float(row[4]), 
                                               float(row[5]), 
                                               types))

now we have a database of every point in every conformation

database == list of molecules = \[molecule, ..., molecule\]

molecule == list of conformations = \[conformation, ..., conformation\]

conformation == list of points \[point, ..., point\]

database = \[ \[ \[ Points \] \], ..., \[ \[ Points \] \] \]

In [ ]:
for idxMol, mol in enumerate(database):
    print("new molecule")
    for idxConf, conf in enumerate(mol):
        print("new conformation")
        for idxPoint, point in enumerate(conf):
            print (point.x,point.y,point.z,point.data)

### Auxiliary functions for restriction verification

The first definition of the data set uses the distance (d) between every two points, $(x_1,y_1,z_1)$ and $(x_2,y_2,z_2)$.

$$
d = ((x_2 - x_1)^2 + (y_2 - y_1)^2 + (z_2 - z_1)^2)^{1/2} 
$$

In [ ]:
def dist ( PointA, PointB ):
    xa = PointA.x
    xb = PointB.x
    ya = PointA.y
    yb = PointB.y
    za = PointA.z
    zb = PointB.z 

    final = m.sqrt( (xa-xb)**2 + (ya-yb)**2 + (za-zb)**2 )

    return final

In [ ]:
def sameType (*points): #return true or false whenever the 2 or more points are of the same type or not
    result = True
    type_data = ""
    
    if(len(points)<2):
        raise Exception('Number of points to be compared should be at least 2. The number of points was: {}'.format(len(points)))
    
    for i in points:
        if (type_data == ""):
            type_data = i.data
        
        result = result and (type_data==i.data)
        
    return result

In [ ]:
def distances (*points): 
    # returns the distance between 2 or more points of the same type, if 
    # the alignment between them is possible
    value = 0
    pairs = [] #list with the distances between each pairs

    if sameType(*points):
        for pair in comb(points,2):
            pairs.append(dist(pair[0],pair[1]))

    if (all(dists<=2 for dists in pairs )):
        value = sum(pairs)
    
    return value

## Pre-Alignment

In [ ]:
def convertTolist(conf):
    
    c = []
    t = []
    
    for idx, p in enumerate(conf):
        c.append([])
        c[idx].append(p.x)
        c[idx].append(p.y)
        c[idx].append(p.z)
        t.append(p.data)
    
    return [np.array(c),t]

def unconvert(c,t):
    conf = []
    
    for idx, p in enumerate(c):
        conf.append(Point(p[0],p[1],p[2],t[idx]))
        
    
    return conf

In [ ]:
def besterror(c1, c2, c3):
    
    combinations = []
    best = -1
    pos = 0
    
    #first
    reg1 = rigid_registration(**{ 'X': c1, 'Y': c2 })
    reg2 = rigid_registration(**{ 'X': c1, 'Y': c3 })
    combinations.append([reg1.register()[2] + reg2.register()[2],
                         c1,
                         reg1.register()[0],
                         reg2.register()[0]])
    
    #second
    reg1 = rigid_registration(**{ 'X': c2, 'Y': c1 })
    reg2 = rigid_registration(**{ 'X': c2, 'Y': c3 })
    combinations.append([reg1.register()[2] + reg2.register()[2],
                         reg1.register()[0],
                         c2,
                         reg2.register()[0]])
    
    #third
    reg1 = rigid_registration(**{ 'X': c3, 'Y': c1 })
    reg2 = rigid_registration(**{ 'X': c3, 'Y': c2 })
    combinations.append([reg1.register()[2] + reg2.register()[2],
                         reg1.register()[0],
                         reg2.register()[0],
                         c3])
    
    #fourth
    reg1 = rigid_registration(**{ 'X': c1, 'Y': c2 })
    reg2 = rigid_registration(**{ 'X': reg1.register()[0], 'Y': c3 })
    combinations.append([reg1.register()[2] + reg2.register()[2],
                         c1, 
                         reg1.register()[0],
                         reg2.register()[0]])
    
    #fifth
    reg1 = rigid_registration(**{ 'X': c1, 'Y': c3 })
    reg2 = rigid_registration(**{ 'X': reg1.register()[0], 'Y': c2 })
    combinations.append([reg1.register()[2] + reg2.register()[2],
                         c1,
                         reg2.register()[0],
                         reg1.register()[0]])
    
    #sixth
    reg1 = rigid_registration(**{ 'X': c2, 'Y': c1 })
    reg2 = rigid_registration(**{ 'X': reg1.register()[0], 'Y': c3 })
    combinations.append([reg1.register()[2] + reg2.register()[2],
                         reg1.register()[0],
                         c2,
                         reg2.register()[0]])
    
    #seventh
    reg1 = rigid_registration(**{ 'X': c2, 'Y': c3 })
    reg2 = rigid_registration(**{ 'X': reg1.register()[0], 'Y': c1 })
    combinations.append([reg1.register()[2] + reg2.register()[2],
                         reg2.register()[0],
                         c2,
                         reg1.register()[0]])
    
    #eighth
    reg1 = rigid_registration(**{ 'X': c3, 'Y': c1 })
    reg2 = rigid_registration(**{ 'X': reg1.register()[0], 'Y': c2 })
    combinations.append([reg1.register()[2] + reg2.register()[2],
                         reg1.register()[0],
                         reg2.register()[0],
                         c3])
    
    #ninth
    reg1 = rigid_registration(**{ 'X': c3, 'Y': c2 })
    reg2 = rigid_registration(**{ 'X': reg1.register()[0], 'Y': c1 })
    combinations.append([reg1.register()[2] + reg2.register()[2],
                         reg2.register()[0],
                         reg1.register()[0],
                         c3])
    
    
    for idx, elem in enumerate(combinations):
        
        if(best==-1 or elem[0]<best ):
            best = elem[0]
            pos = idx
    
    return combinations[pos][1], combinations[pos][2], combinations[pos][3] 
    

In [ ]:
def transformation(*confs):
    # X and Y are lists with coordinates of 2 different
    #conformations that we want to "align"
    if (len(confs)==2):
        [c1,t1], [c2,t2] = convertTolist(confs[0]), convertTolist(confs[1])
        
        reg1 = rigid_registration(**{ 'X': c1, 'Y': c2 })
        TY1, _, err1 = reg1.register()
    
        reg2 = rigid_registration(**{ 'X': c2, 'Y': c1 })
        TY2, _, err2 = reg2.register()
        
        if(err1 < err2):
            result = unconvert(c1,t1), unconvert(TY1,t2)
        else:
            result = unconvert(TY2,t1), unconvert(c2,t2)
        
    if (len(confs)==3):
        [c1,t1], [c2,t2], [c3,t3] = convertTolist(confs[0]), convertTolist(confs[1]), convertTolist(confs[2])
        
        nc1, nc2, nc3 = besterror(c1,c2,c3)
        
        result = unconvert(nc1,t1), unconvert(nc2,t2), unconvert(nc3,t3)
    
    return result

## SAT solver for 3 molecules

In [ ]:
def solve3(conf1,conf2,conf3,alignN):
       
    aviso = 0
    
    pmol1 = len(conf1)
    pmol2 = len(conf2)
    pmol3 = len(conf3)
    
    cmol1 = conf1 # molecule 1 conformation
    cmol2 = conf2 # molecule 2 conformation
    cmol3 = conf3 # molecule 3 conformation
    
    size = pmol1*pmol2*pmol3  #number of elements of matrix solution

    toReturn = np.zeros(size) 
    
    try:
        m = Model("trial")
        m.Params.OutputFlag = 0
        elems = m.addVars(pmol1,pmol2,pmol3,vtype=GRB.BINARY) #array with 1 or 0
        dists = m.addVars(pmol1,pmol2,pmol3,lb=0.0,ub=60001.0,vtype=GRB.INTEGER) # array with elements (distances)

        
        for id1, p1 in enumerate(cmol1):
            for id2, p2 in enumerate(cmol2):
                for id3, p3 in enumerate(cmol3):
                    
                    value = round(10000*distances(p1,p2,p3),0)
                    
                    if ( not(sameType(p1,p2,p3)) or value == 0 ):
                        m.addConstr( elems[id1,id2,id3] == 0 )
                    
                    m.addConstr( (elems[id1,id2,id3] == 0) >> (dists[id1,id2,id3] == 0.0) )
                    m.addConstr( (elems[id1,id2,id3] == 1) >> (dists[id1,id2,id3] == value) )
    
        # each point form conf1 can only be connected to one point from each conformation 
        for p1 in range(pmol1):
            m.addConstr( sum( elems[p1,p2,p3] for p2 in range(pmol2) for p3 in range(pmol3)) <=1 )
            
        for p2 in range(pmol2):
            m.addConstr( sum( elems[p1,p2,p3] for p1 in range(pmol1) for p3 in range(pmol3)) <=1 )
        
        for p3 in range(pmol3):
            m.addConstr( sum( elems[p1,p2,p3] for p1 in range(pmol1) for p2 in range(pmol2)) <=1 )

        m.addConstr( elems.sum() >= alignN ) 
        #relevant pharmacophore has to have at least 3 commun groups
        
               
        m.setObjective( elems.sum(), GRB.MAXIMIZE)
        m.setObjective( dists.sum(), GRB.MINIMIZE)
               

        m.optimize()
        
        aviso = 0
        
        if (m.status == GRB.Status.OPTIMAL):
            solD = m.getAttr('X', dists)
            for i in range(pmol1):
                for j in range(pmol2):
                    for k in range(pmol3):
                        toReturn[i + pmol1*( j + pmol2*k)] = solD[i,j,k]
        elif (m.status == GRB.Status.INFEASIBLE):
            aviso=1
                    
    except GurobiError as e:
        print('Error code ' + str(e.errno) + ": " + str(e))
        aviso = -1

    except AttributeError:
        aviso = -1
        print('Encountered an attribute error')

    return [aviso,toReturn]


## SAT solver for 2 molecules

In [ ]:
def solve2(conf1,conf2,alignN):
       
    aviso = 0
    
    pmol1 = len(conf1)
    pmol2 = len(conf2)
    
    cmol1 = conf1 # molecule 1 conformation
    cmol2 = conf2 # molecule 2 conformation
    
    size = pmol1*pmol2  #number of elements of matrix solution

    toReturn = np.zeros(size) 
    
    try:
        m = Model("trial")
        m.Params.OutputFlag = 0
        elems = m.addVars(pmol1,pmol2,vtype=GRB.BINARY) #array with 1 or 0
        dists = m.addVars(pmol1,pmol2,lb=0.0,ub=20001.0,vtype=GRB.INTEGER) # array with elements (distances)

        
        for id1, p1 in enumerate(cmol1):
            for id2, p2 in enumerate(cmol2):
                    
                    value = round(10000*distances(p1,p2),0)
                    
                    if ( not(sameType(p1,p2)) or value == 0 ):
                        m.addConstr( elems[id1,id2] == 0 )
                    
                    m.addConstr( (elems[id1,id2] == 0) >> (dists[id1,id2] == 0.0) )
                    m.addConstr( (elems[id1,id2] == 1) >> (dists[id1,id2] == value) )
    
        # each point from conf1 can only be connected to one point from each conformation 
        for p1 in range(pmol1):
            m.addConstr( sum( elems[p1,p2] for p2 in range(pmol2)) <=1 )
            
        for p2 in range(pmol2):
            m.addConstr( sum( elems[p1,p2] for p1 in range(pmol1)) <=1 )
        
        m.addConstr( elems.sum() >= alignN) #relevant pharmacophore has to have at least 3 commun groups
        
               
        m.setObjective( elems.sum(), GRB.MAXIMIZE)
        m.setObjective( dists.sum(), GRB.MINIMIZE)
               

        m.optimize()
        
        aviso = 0
        
        if (m.status == GRB.Status.OPTIMAL):
            solD = m.getAttr('X', dists)
            for i in range(pmol1):
                for j in range(pmol2):
                        toReturn[i + pmol1*j ] = solD[i,j]
        elif (m.status == GRB.Status.INFEASIBLE):
            aviso=1
                    
    except GurobiError as e:
        print('Error code ' + str(e.errno) + ": " + str(e))
        aviso = -1

    except AttributeError:
        aviso = -1
        print('Encountered an attribute error')

    return [aviso,toReturn]


Auxiliary functions to deal with data input and output of the classical SAT solver. Used below.

In [ ]:
def decompose3(numberPoints,number,a,b):
    n1 = numberPoints[0] 
    n2 = numberPoints[1] 
    n3 = numberPoints[2]
    n4=n1*n2 
    
    if(number>=n4):
        b += 1
        return decompose3(numberPoints,number-n4,a,b)
    else:
        if(number>=n1):
            a += 1
            return decompose3(numberPoints,number-n1,a,b)
        else:
            return [number,a,b]

In [ ]:
def decompose2(numberPoints,number,a):
    n1 = numberPoints[0] #7 or 5
    n2 = numberPoints[1] #5
    
    if(number>=n1):
        a += 1
        return decompose2(numberPoints,number-n1,a)
    else:
        return [number,a]

In [ ]:
def toMatrix(lista,numberPoints):
    matrix = np.zeros(tuple(numberPoints))
    nmols = len(numberPoints)
    
    if(nmols==3): 
        for idx, elem in enumerate(lista):
            [id1,id2,id3] = decompose3(numberPoints,idx,0,0)
            matrix[id1][id2][id3] = elem
            
    if(nmols==2):
        for idx, elem in enumerate(lista):
            [id1,id2] = decompose2(numberPoints,idx,0)
            matrix[id1][id2] = elem
    
    return matrix

## Creation of the solution structures

In [ ]:
conf_sol = []
def combinations (*molecules):
    
    numberPoints = []
    for mol in molecules:
        numberPoints.append(len(mol[0]))
    
    listmatrix = []
    matrix_list1 = []
    matrix_list2 = []
    
    newList = []
    
    size = len(molecules)
    
    if(size==3):
        for idxConf1, conf1 in enumerate(molecules[0]):
            for idxConf2, conf2 in enumerate(molecules[1]):
                for idxConf3, conf3 in enumerate(molecules[2]):
                    
                    nconf1, nconf2, nconf3 = transformation(conf1,conf2,conf3)
                    
                    conf_sol.append([nconf1, nconf2, nconf3]) #new conformation coordinates

                    [aviso, matrix_list1] = solve3(nconf1,nconf2,nconf3,3)
                    
                    if aviso==0:
                        [aviso, matrix_list2] = solve3(nconf1,nconf2,nconf3,4)
                        
                        if aviso==0:
                            matrix_list1 = []
                            [aviso, matrix_list1] = solve3(nconf1,nconf2,nconf3,5)
                            
                            if aviso==0:
                                newList = [x/10000 for x in matrix_list1]
                                listmatrix.append( toMatrix(newList,numberPoints)  )
                            elif aviso==1:
                                newList = [x/10000 for x in matrix_list2]
                                listmatrix.append( toMatrix(newList,numberPoints)  )
                        
                        elif aviso==1:
                            newList = [x/10000 for x in matrix_list1]
                            listmatrix.append( toMatrix(newList,numberPoints)  )
                    
                    elif aviso==1:
                        matrix = np.zeros(tuple(numberPoints))
                        listmatrix.append(matrix)
                    
                    if(aviso==-1):
                        print("ERROR IN SAT SOLVER")

                    matrix_list1 = []
                    matrix_list2 = []
                    newList = []
                
    if(size==2):
        for idxConf1, conf1 in enumerate(molecules[0]):
            for idxConf2, conf2 in enumerate(molecules[1]):
                
                nconf1, nconf2 = transformation(conf1,conf2)
                conf_sol.append([nconf1, nconf2]) #new conformation coordinates
                
                [aviso, matrix_list1] = solve2(nconf1,nconf2,3)

                if aviso==0:
                    [aviso, matrix_list2] = solve2(nconf1,nconf2,4)

                    if aviso==0:
                        matrix_list1 = []
                        [aviso, matrix_list1] = solve2(nconf1,nconf2,5)

                        if aviso==0:
                            newList = [x/10000 for x in matrix_list1]
                            listmatrix.append( toMatrix(newList,numberPoints)  )
                        elif aviso==1:
                            newList = [x/10000 for x in matrix_list2]
                            listmatrix.append( toMatrix(newList,numberPoints)  )

                    elif aviso==1:
                        newList = [x/10000 for x in matrix_list1]
                        listmatrix.append( toMatrix(newList,numberPoints)  )

                elif aviso==1:
                    matrix = np.zeros(tuple(numberPoints))
                    listmatrix.append(matrix)

                if(aviso==-1):
                    print("ERROR IN SAT SOLVER")

                matrix_list1 = []
                matrix_list2 = []
                newList = []
    
    return listmatrix

### Assign Score to solution

Score of each solution is given by:


$$\text{score}(n_{aligned},d) = round\left( \frac{200 \times d}{\binom{M}{2} \times n_{aligned}} + 400 \times (5-n_{aligned}) \right)$$

where $M$ is the number of Molecules to align, $n_{aligned}$ is the number of points aligned and $d$ is the sum of distances of the connected points

In [ ]:
def NonZeroAndSum(matrix):
    nz=0
    s=0
    
    for idC1, c1 in enumerate(matrix):
        for idC2, c2 in enumerate(c1):
            if isinstance(c2, (np.ndarray)): #tests if this is an array or not
                for idC3, c3 in enumerate(c2):
                    if (c3!=0):
                        s += c3 
                        nz += 1
            else:
                if (c2!=0):
                    s += c2 
                    nz += 1

    
    return [nz,s] #number of non-zeros; sum of all non-zero elements

In [ ]:
def Scores(listMatrix,M):
    
    scores = []
    
    for matrix in listMatrix:
        naligned, d = NonZeroAndSum(matrix)
        if (naligned!=0):
            scores.append( int(round( (d*200/(naligned*comb2(M,2))) + (5-naligned)*400 )) )
        else:
            scores.append(int(2000))
    
    return scores

Auxiliary functions to find correspondence between position in the list and conformations from each molecule.

In [ ]:
def which2(number,a):
    if(number>=12):
        a += 1
        return which2(number-12,a)
    else:
        print("Conformation from molecule given as input 1: ",a,
              "\nConformation from molecule given as input 2: ",number)
        return [a, number] 

def which3(number,a,b):
    if (number>=144):
        a += 1
        return which3(number-144,a,b)
    else:
        if(number>=12):
            b += 1
            return which3(number-12,a,b)
        else:
            print("Conformation from molecule given as input 1: ",a,
                  "\nConformation from molecule given as input 2: ",b,
                  "\nConformation from molecule given as input 3: ",number)
            return [a, b, number]
        
def whichConfs (number,mol): 
    #insert position of listmatrix and returns to each conformations that correspond
    if(mol==3):
        numbers = which3(number,0,0)
    if(mol==2):
        numbers = which2(number,0)
    return numbers

## Classic Search

Print the solution with the most connections and least sum of the distance of this connections.

In [ ]:
def answer (scores):
    bestSum = max(scores)+1 #'random' high number
    solution = -1
    
    sols = []
    vals = []

    for idS, s in enumerate(scores):
        if( s < bestSum):
            bestSum = s
            solution = idS
            print(s, idS)
    
    print("------------------")
    print("Position of solution:",solution) #position in array listmatrix/scores
    print("------------------")
    
    return solution, bestSum #position of solution, value of solution

# Solutions

In [ ]:
print("\nMolecula 1, 2 e 3")
mol123 = combinations(database[0],database[1],database[2])
scores123 = Scores(mol123,3)
pos123, value = answer(scores123)
whichConfs(pos123,3)

In [ ]:
print("\nMolecula 1 e 2")
mol12 = combinations(database[0],database[1]) 
scores12 = Scores(mol12,2)
pos12, value = answer(scores12)
whichConfs(pos12,2)

In [ ]:
print("\nMolecula 1 e 3")
mol13 = combinations(database[0],database[2])
scores13 = Scores(mol13,2)
pos13, value = answer(scores13)
whichConfs(pos13,2)

In [ ]:
print("\nMolecula 2 e 3")
mol23 = combinations(database[1],database[2])
scores23 = Scores(mol23,2)
pos23, value = answer(scores23)
whichConfs(pos23,2)

## Store Solution's Matrixes

In [ ]:
with open('mol123.txt', 'w') as f:
    f.write("Molecules 1, 2 and 3\n")
    for idx, item in enumerate(mol123):
        [a, b, c] = whichConfs(idx,3)
        f.write("Conformations %s, %s and %s, from respective molecules.\n"% (a,b,c))
        f.write("%s\n" % item)
f.close()

In [ ]:
with open('mol12.txt', 'w') as f:
    f.write("Molecules 1 and 2\n")
    for idx, item in enumerate(mol12):
        [a,b] = whichConfs(idx,2)
        f.write("Conformations %s and %s, from respective molecules.\n" % (a, b))
        f.write("%s\n" % item)
f.close()

In [ ]:
with open('mol13.txt', 'w') as f:
    f.write("Molecules 1 and 3\n")
    for idx, item in enumerate(mol13):
        [a,b] = whichConfs(idx,2)
        f.write("Conformations %s and %s, from respective molecules.\n" % (a, b))
        f.write("%s\n" % item)
f.close()

In [ ]:
with open('mol23.txt', 'w') as f:
    f.write("Molecules 2 and 3\n")
    for idx, item in enumerate(mol23):
        [a,b] = whichConfs(idx,2)
        f.write("Conformations %s and %s, from respective molecules.\n" % (a, b))
        f.write("%s\n" % item)
f.close()

## Store Scores

In [ ]:
with open('scores12.txt', 'w') as f:
    for item in scores12:
        f.write("%s\n" % item)
f.close()

In [ ]:
with open('scores13.txt', 'w') as f:
    for item in scores13:
        f.write("%s\n" % item)
f.close()

In [ ]:
with open('scores23.txt', 'w') as f:
    for item in scores23:
        f.write("%s\n" % item)
f.close()

In [ ]:
with open('scores123.txt', 'w') as f:
    for item in scores123:
        f.write("%s\n" % item)
f.close()